In [1]:
import pandas as pd
import os
import csv
import numpy as np

In [6]:
def average(data):
    df=data
    
    # Convertir la columna 'timestamp_CPU' a datetime si es necesario
    df['timestamp_CPU'] = pd.to_datetime(df['timestamp_CPU'], unit='s')

    # Redondear los timestamps al segundo más cercano
    df['timestamp_CPU'] = df['timestamp_CPU'].dt.round('S')

    # Agrupar los datos por 'timestamp_CPU' y calcular la media de las demás columnas
    grouped_data = df.groupby('timestamp_CPU').mean().reset_index()

    return grouped_data
    


In [7]:
def find_delimiter(filename): #detector del separador que contiene el archivo CSV
    sniffer= csv.Sniffer()
    with open(filename) as fp:
        delimiter = sniffer.sniff(fp.read(5000)).delimiter
    return delimiter

In [8]:
def clean_data(input_file, output_file):
    #Apertura del archivo con Pandas DataFrame 
    #Detección del separador de datos
    separador= find_delimiter(input_file)
    df=pd.read_csv(input_file, sep= separador)

    headers = df.columns
    filac = df.iloc[0]

    #Eliminación de filas con NaN y realización de la media según las timestamp
    clean_df = average(df.dropna(axis=0))
    
    #Escribe un los datos filtrados en un archivo nuevo 
    clean_df.to_csv(output_file, index=False, sep = separador)
    print("Datos limpios guardados como ", output_file)

In [9]:
def main(): 
    while True: 
        # Solicitar al usuario que ingrese el nombre del archivo de entrada
        input_file = input("Ingrese la ruta y nombre del archivo CSV de entrada: ")
    
         # Solicitar al usuario que ingrese el nombre del archivo de salida
        output_file = input("Ingrese la ruta y el nombre del archivo CSV de salida para los datos limpios: ")
    
        if input_file == output_file: 
            print("Por favor, indique nombres para los archivos que sean distintivos")
        else: 
            # Llamar a la función para limpiar los datos
            clean_data(input_file, output_file)
            break

In [6]:
if __name__ == "__main__":
    main()

Ingrese la ruta y nombre del archivo CSV de entrada:  datosCSVleer/001_shimmer_data.csv
Ingrese la ruta y el nombre del archivo CSV de salida para los datos limpios:  datosCSVleer/001_shimmer_limpios.csv


Datos limpios guardados como  datosCSVleer/001_shimmer_limpios.csv


C:\Users\isagl\AppData\Local\Temp\ipykernel_23252\2979926440.py:8: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['timestamp_CPU'] = df['timestamp_CPU'].dt.round('S')
